In [5]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, Checkbox, HBox, VBox, Label, interactive_output


# ---------- Beregningsfunksjon ----------
def mode_response(omega_n, zeta, A_n, t):
    """Returnerer y_n(t) og amplitudekappe (kun for underdempet tilfelle)."""
    if zeta < 1.0:  # underdempet
        omega_d = omega_n * np.sqrt(1.0 - zeta**2)
        A = A_n
        B = (zeta * omega_n * A_n) / omega_d
        y = np.exp(-zeta * omega_n * t) * (A*np.cos(omega_d*t) + B*np.sin(omega_d*t))
        env = A_n * np.exp(-zeta * omega_n * t)  # positiv kappedemping
    elif np.isclose(zeta, 1.0, atol=1e-9):  # kritisk dempet
        A = A_n
        B = omega_n * A_n
        y = (A + B*t) * np.exp(-omega_n * t)
        env = None
    else:  # overdempet
        s = np.sqrt(zeta**2 - 1.0)
        r1 = -omega_n * (zeta + s)
        r2 = -omega_n * (zeta - s)
        A = A_n * r2 / (r2 - r1)
        B = -A_n * r1 / (r2 - r1)
        y = A*np.exp(r1*t) + B*np.exp(r2*t)
        env = None
    return y, env


# ---------- Hovedplot ----------
def mass_spring_harmonics(m, k, c, x0, v0_unused, n_periods, show_decay):
    omega0 = np.sqrt(k/m)
    zeta = c / (2.0*np.sqrt(m*k))
    omega_d = omega0*np.sqrt(1.0 - zeta**2) if zeta < 1.0 else None

    T0 = 2*np.pi / omega0
    t_end = n_periods * T0
    t = np.linspace(0, t_end, 4000)

    colors = plt.cm.viridis(np.linspace(0,1,5))
    plt.figure(figsize=(9,5))

    for n, col in zip(range(0,5), colors):
        omega_n = (n+1)*omega0
        A_n = x0/(n+1)

        if show_decay:
            y_n, env = mode_response(omega_n, zeta, A_n, t)
        else:
            y_n, env = A_n*np.cos(omega_n*t), None

        # signal
        plt.plot(t/T0, y_n, color=col, lw=1.6,
                 label=f"n={n}, ω={omega_n:.2f} rad/s ({omega_n/(2*np.pi):.2f} Hz)")

        # envelope kun for underdempet tilfelle (positiv)
        if (env is not None) and (zeta < 1.0):
            plt.plot(t/T0, env, color=col, ls="--", lw=1)

    # Regimebeskrivelse
    if zeta < 1:
        regime = "Underdempet"
        freq_info = f"ωd={omega_d:.2f} rad/s ({omega_d/(2*np.pi):.2f} Hz)"
        subtitle = ""
    elif np.isclose(zeta, 1.0, atol=1e-9):
        regime = "Kritisk dempet"
        freq_info = "Ingen oscillasjon (aperiodisk retur)"
        subtitle = ""
    else:
        regime = "Overdempet"
        freq_info = "Ingen oscillasjon (to eksponentielle forløp)"
        s = np.sqrt(zeta**2 - 1.0)
        r1 = -omega0*(zeta + s)
        r2 = -omega0*(zeta - s)
        subtitle = f"r₁={r1:.2f}, r₂={r2:.2f} [1/s]"

    plt.title(
        f"ζ={zeta:.3f} | ω₀={omega0:.2f} rad/s ({omega0/(2*np.pi):.2f} Hz) | {freq_info}\n"
        f"{regime} | {'Med demping' if show_decay else 'Uten demping'}",
        fontsize=11
    )
    if subtitle:
        plt.suptitle(subtitle, fontsize=9, y=0.96)

    plt.xlabel("Tid [antall perioder T₀]")
    plt.ylabel("Amplitude (normalisert)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# ---------- Sliders i to kolonner ----------
slider_left = VBox([
    Label(value="🧩 Systemparametre"),
    FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='m [kg]'),
    FloatSlider(value=100.0, min=10.0, max=500.0, step=10.0, description='k [N/m]'),
    FloatSlider(value=0.1, min=0.0, max=10.0, step=0.1, description='c [Ns/m]')
])

slider_right = VBox([
    Label(value="🎛️ Visningskontroll"),
    FloatSlider(value=0.01, min=0.0, max=0.05, step=0.001, description='x₀ [m]'),
    FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='v₀ [m/s]'),
    IntSlider(value=5, min=1, max=20, step=1, description='Antall perioder'),
    Checkbox(value=True, description='Vis demping')
])

ui = HBox([slider_left, slider_right])

# ---------- Koble widgets og funksjon ----------
out = interactive_output(
    mass_spring_harmonics,
    {
        'm': slider_left.children[1],
        'k': slider_left.children[2],
        'c': slider_left.children[3],
        'x0': slider_right.children[1],
        'v0_unused': slider_right.children[2],
        'n_periods': slider_right.children[3],
        'show_decay': slider_right.children[4]
    }
)

display(ui, out)

Output()